In [1]:
# First attempt to get the data
# https://suche.transparenz.hamburg.de/dataset/verkehrsdaten-rad-infrarotdetektoren-hamburg15

import requests
import pandas as pd
from time import sleep
from random import randint

# Initial API call
base_url = 'https://iot.hamburg.de/v1.0/Things?$filter=Datastreams/properties/serviceName%20eq%20%27HH_STA_HamburgerRadzaehlnetz%27%20and%20Datastreams/properties/layerName%20eq%20%27Anzahl_Fahrraeder_Zaehlstelle_1-Stunde%27&$count=true&$expand=Datastreams($filter=properties/layerName%20eq%20%27Anzahl_Fahrraeder_Zaehlstelle_1-Stunde%27;$expand=Observations($orderby=phenomenonTime%20desc))'

def fetch_data(url):
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Request failed with status code: {response.status_code}")
    return response.json()

def extract_data(json_data):
    rows = []
    things = json_data.get('value', [])
    
    for thing in things:
        thing_id = thing['@iot.id']
        thing_name = thing['name']
        thing_description = thing['description']
        
        for datastream in thing['Datastreams']:
            datastream_id = datastream['@iot.id']
            datastream_name = datastream['name']
            datastream_description = datastream['description']
            
            for observation in datastream['Observations']:
                row = {
                    'ThingID': thing_id,
                    'ThingName': thing_name,
                    'ThingDescription': thing_description,
                    'DatastreamID': datastream_id,
                    'DatastreamName': datastream_name,
                    'DatastreamDescription': datastream_description,
                    'ObservationID': observation['@iot.id'],
                    'PhenomenonTime': observation['phenomenonTime'],
                    'ResultTime': observation['resultTime'],
                    'Result': observation['result']
                }
                rows.append(row)
    return rows

# Fetch initial data
json_data = fetch_data(base_url)
all_data = extract_data(json_data)

# Handle pagination
while '@iot.nextLink' in json_data:
    next_url = json_data['@iot.nextLink']
    sleep(randint(1,3000)/1000)  # Add a 1-second delay between requests
    json_data = fetch_data(next_url)
    all_data.extend(extract_data(json_data))

# Create DataFrame
data = pd.DataFrame(all_data)
print(data.head())

   ThingID                   ThingName  \
0     5564  Verkehrszählstelle 0295970   
1     5564  Verkehrszählstelle 0295970   
2     5564  Verkehrszählstelle 0295970   
3     5564  Verkehrszählstelle 0295970   
4     5564  Verkehrszählstelle 0295970   

                                    ThingDescription  DatastreamID  \
0  Verkehrszählstelle zur Zählung der vom Infraro...         11797   
1  Verkehrszählstelle zur Zählung der vom Infraro...         11797   
2  Verkehrszählstelle zur Zählung der vom Infraro...         11797   
3  Verkehrszählstelle zur Zählung der vom Infraro...         11797   
4  Verkehrszählstelle zur Zählung der vom Infraro...         11797   

                                      DatastreamName  \
0  Fahrradaufkommen an Verkehrszählstelle 0295970...   
1  Fahrradaufkommen an Verkehrszählstelle 0295970...   
2  Fahrradaufkommen an Verkehrszählstelle 0295970...   
3  Fahrradaufkommen an Verkehrszählstelle 0295970...   
4  Fahrradaufkommen an Verkehrszählstelle 0295

## Final code

In [2]:
import requests
import pandas as pd
from time import sleep
import datetime
from random import randint

# Funktion zum Abrufen der Zählstellen
def fetch_things(url):
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for HTTP errors
    return response.json()

# Basis-URL zum Abrufen der Zählstellen mit dem spezifischen Datastream für tägliche Werte
base_url = (
    "https://iot.hamburg.de/v1.0/Things?"
    "$filter=Datastreams/properties/serviceName eq 'HH_STA_HamburgerRadzaehlnetz' "
    "and Datastreams/properties/layerName eq 'Anzahl_Fahrraeder_Zaehlstelle_1-Tag'&"
    "$expand=Datastreams($filter=properties/layerName eq 'Anzahl_Fahrraeder_Zaehlstelle_1-Tag')"
)

# Zählstellen abrufen
things_data = fetch_things(base_url)
things = things_data['value']

# Weitere Seiten abrufen, falls vorhanden
while '@iot.nextLink' in things_data:
    next_url = things_data['@iot.nextLink']
    sleep(randint(1,3000)/1000)  # Respektvolle Pause zwischen den Anfragen
    things_data = fetch_things(next_url)
    things.extend(things_data['value'])

# Alle Datastreams extrahieren
datastreams = []
for thing in things:
    for datastream in thing['Datastreams']:
        datastreams.append(datastream)

print(f"Anzahl der Zählstellen: {len(things)}")
print(f"Anzahl der Datastreams: {len(datastreams)}")


Anzahl der Zählstellen: 160
Anzahl der Datastreams: 160


In [3]:
def fetch_observations(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

# Aktuelles Datum und Datum vor einem Jahr
end_date = datetime.datetime.utcnow()
start_date = end_date - datetime.timedelta(days=365)

# Daten für alle Datastreams abrufen
all_data = []

for datastream in datastreams:
    datastream_id = datastream['@iot.id']
    observations_url = (
        f"https://iot.hamburg.de/v1.0/Datastreams({datastream_id})/Observations?"
        f"$filter=phenomenonTime ge {start_date.isoformat()}Z and phenomenonTime le {end_date.isoformat()}Z&"
        "$orderby=phenomenonTime desc"
    )
    
    observations_data = fetch_observations(observations_url)
    observations = observations_data['value']
    
    while '@iot.nextLink' in observations_data:
        next_url = observations_data['@iot.nextLink']
        sleep(randint(1,3000)/1000)  # Respektvolle Pause zwischen den Anfragen
        observations_data = fetch_observations(next_url)
        observations.extend(observations_data['value'])
    
    # Extrahieren und speichern der relevanten Daten
    for observation in observations:
        row = {
            'DatastreamID': datastream_id,
            'PhenomenonTime': observation['phenomenonTime'],
            'ResultTime': observation['resultTime'],
            'Result': observation['result']
        }
        all_data.append(row)

# In ein DataFrame konvertieren
data = pd.DataFrame(all_data)
print(data.head())




   DatastreamID                             PhenomenonTime  \
0         11798  2024-05-25T22:00:00Z/2024-05-26T21:59:59Z   
1         11798  2024-05-24T22:00:00Z/2024-05-25T21:59:59Z   
2         11798  2024-05-23T22:00:00Z/2024-05-24T21:59:59Z   
3         11798  2024-05-22T22:00:00Z/2024-05-23T21:59:59Z   
4         11798  2024-05-21T22:00:00Z/2024-05-22T21:59:59Z   

                 ResultTime  Result  
0  2024-05-27T00:54:18.399Z    1397  
1  2024-05-26T00:54:19.385Z    2769  
2  2024-05-25T00:54:28.749Z    2449  
3  2024-05-24T00:54:19.968Z    1621  
4  2024-05-23T00:54:24.515Z    1658  


In [4]:
data

,DatastreamID,PhenomenonTime,ResultTime,Result
0,11798,2024-05-25T22:00:00Z/2024-05-26T21:59:59Z,2024-05-27T00:54:18.399Z,1397
1,11798,2024-05-24T22:00:00Z/2024-05-25T21:59:59Z,2024-05-26T00:54:19.385Z,2769
2,11798,2024-05-23T22:00:00Z/2024-05-24T21:59:59Z,2024-05-25T00:54:28.749Z,2449
3,11798,2024-05-22T22:00:00Z/2024-05-23T21:59:59Z,2024-05-24T00:54:19.968Z,1621
4,11798,2024-05-21T22:00:00Z/2024-05-22T21:59:59Z,2024-05-23T00:54:24.515Z,1658
...,...,...,...,...
58127,18404,2023-06-01T22:00:00Z/2023-06-02T21:59:59Z,2024-03-13T23:43:46.131Z,1693
58128,18404,2023-05-31T22:00:00Z/2023-06-01T21:59:59Z,2024-03-13T23:40:25.458Z,1719
58129,18404,2023-05-30T22:00:00Z/2023-05-31T21:59:59Z,2024-03-13T23:37:19.738Z,1974
58130,18404,2023-05-29T22:00:00Z/2023-05-30T21:59:59Z,2024-03-13T23:34:16.291Z,1756


In [5]:
data.to_csv('yearly_bike_data_daily.csv', index=False)

In [6]:
# Extracting the required information from each dictionary in the list
details = pd.DataFrame([{
    'Asset ID': thing['properties']['assetID'],
    'Datastream ID': thing['Datastreams'][0]['@iot.id'],
    'Latitude': thing['Datastreams'][0]['observedArea']['coordinates'][1],
    'Longitude': thing['Datastreams'][0]['observedArea']['coordinates'][0]
   } for thing in things])

In [7]:
details

,Asset ID,Datastream ID,Latitude,Longitude
0,0295970,11798,"[9.999282, 53.580056]","[9.999377, 53.580126]"
1,0295971,11802,"[9.999252, 53.580071]","[9.999336767, 53.580143453]"
2,0295972,11806,"[9.999314, 53.580039]","[9.999405, 53.580105]"
3,0295910,11822,53.580518,9.999531
4,0295911,11826,53.580504,9.99951
...,...,...,...,...
155,7435961,18250,53.655824,10.093107
156,7435940,18254,53.656299,10.094785
157,7435942,18258,53.656175,10.094757
158,0109930,18400,53.571422,10.064983


In [8]:
# we need to get rid of the lists inside the lat / long column 

details['Latitude'] = details['Latitude'].apply(lambda x: x[1] if isinstance(x, list) and len(x) > 1 else x)
details['Longitude'] = details['Longitude'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 1 else x)

In [9]:
details

,Asset ID,Datastream ID,Latitude,Longitude
0,0295970,11798,53.580056,9.999377
1,0295971,11802,53.580071,9.999337
2,0295972,11806,53.580039,9.999405
3,0295910,11822,53.580518,9.999531
4,0295911,11826,53.580504,9.999510
...,...,...,...,...
155,7435961,18250,53.655824,10.093107
156,7435940,18254,53.656299,10.094785
157,7435942,18258,53.656175,10.094757
158,0109930,18400,53.571422,10.064983


In [10]:
data = data.merge(details, left_on='DatastreamID', right_on='Datastream ID', how='left')

In [11]:
data = data.drop(['Datastream ID'], axis=1)
data

,DatastreamID,PhenomenonTime,ResultTime,Result,Asset ID,Latitude,Longitude
0,11798,2024-05-25T22:00:00Z/2024-05-26T21:59:59Z,2024-05-27T00:54:18.399Z,1397,0295970,53.580056,9.999377
1,11798,2024-05-24T22:00:00Z/2024-05-25T21:59:59Z,2024-05-26T00:54:19.385Z,2769,0295970,53.580056,9.999377
2,11798,2024-05-23T22:00:00Z/2024-05-24T21:59:59Z,2024-05-25T00:54:28.749Z,2449,0295970,53.580056,9.999377
3,11798,2024-05-22T22:00:00Z/2024-05-23T21:59:59Z,2024-05-24T00:54:19.968Z,1621,0295970,53.580056,9.999377
4,11798,2024-05-21T22:00:00Z/2024-05-22T21:59:59Z,2024-05-23T00:54:24.515Z,1658,0295970,53.580056,9.999377
...,...,...,...,...,...,...,...
58127,18404,2023-06-01T22:00:00Z/2023-06-02T21:59:59Z,2024-03-13T23:43:46.131Z,1693,0109932,53.571287,10.065085
58128,18404,2023-05-31T22:00:00Z/2023-06-01T21:59:59Z,2024-03-13T23:40:25.458Z,1719,0109932,53.571287,10.065085
58129,18404,2023-05-30T22:00:00Z/2023-05-31T21:59:59Z,2024-03-13T23:37:19.738Z,1974,0109932,53.571287,10.065085
58130,18404,2023-05-29T22:00:00Z/2023-05-30T21:59:59Z,2024-03-13T23:34:16.291Z,1756,0109932,53.571287,10.065085


In [12]:
data.to_csv('yearly_bike_data_daily.csv', index=False)